In [ ]:
!pip install sentencepiece
!pip install datasets

In [1]:
# Change path if not match your path of shared directory DS
# Make sure that you've mounted with your drive
%cd /content/drive/MyDrive/AI/DS

/content/drive/MyDrive/AI/DS


In [3]:
from transformers import T5EncoderModel, AutoTokenizer, DataCollatorWithPadding

import torch
from torch import nn
from torch.nn import functional as tf
from torch.utils.data import DataLoader

# from datasets import Dataset

import numpy as np
from sklearn.preprocessing import MinMaxScaler

from matplotlib import pyplot as plt

import json
import math
from tqdm import tqdm

from modeling import create_model
from scaler import Scaler

In [ ]:
pretrained_model_name = "VietAI/vit5-base"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
scaler = torch.load("scaler.pt")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = create_model(pretrained_model_name, state_dict_path="checkpoint/model_state_dict.pt", device=device)

In [ ]:
def transform_input(tokenizer, input_dict):
  input_text = f"Loại: {input_dict['estate_type']}{tokenizer.eos_token}Diện tích: {input_dict['square']}{tokenizer.eos_token}Mô tả: {input_dict['description']}"

  return tokenizer(input_text, return_tensors="pt")

In [ ]:
def predict(model, scaler, input_encoding):
  model.eval()
  with torch.no_grad():
    logits = model(**input_encoding)

  prices = scaler.invert(logits.cpu().numpy())
  return prices.tolist()

In [ ]:
transform_input(tokenizer)

In [4]:
data_dir = "data/"

with open(data_dir + "validation_dict.json", "r", encoding="utf-8") as f:
  validation_dict = json.load(f)

with open(data_dir + "test_dict.json", "r", encoding="utf-8") as f:
  test_dict = json.load(f)

In [ ]:
random_index

568

In [6]:
i = 568
input_dict = {}

for k, v in validation_dict.items():
  input_dict[k] = v[i]


In [7]:
input_dict

{'description': 'Bán đất Cổ Linh, gần ngã tư Thạch Bàn, Thế giới di động, Aeonmall khu vực đang phát triển từng ngày Diện tích 140 m2, MT 5,5 m có vỉa hè, đường ô tô tránh, cách mặt đường Cổ Linh khoảng 40 m thích hợp xây tòa nhà kinh doanh, đón đầu tương lai Sổ đỏ full thổ cư, pháp lý chuẩn, sẵn sàng giao dịch Vui lòng liên hệ để biết thêm thông tin chi tiết 0902116316',
 'estate_type': 'Đất',
 'square': 140.0,
 'price': 13300000000.0}

Best indices: [568, 28, 1041, 7498, 4791]

In [ ]:
# random_index = np.random.randint(0, 10000)
selected_indices = []
for i in tqdm(range(len(validation_dict["price"]))):
  input_dict = {}

  for k, v in validation_dict.items():
    input_dict[k] = v[i]

  # print(input_dict["price"])

  input_encoding = transform_input(tokenizer, input_dict)
  prices = predict(model, scaler, input_encoding)

  price = prices[0][0]
  if np.abs(price - input_dict["price"])/input_dict["price"] <= 0.01:
    print("---------------------------")
    print("True:", input_dict["price"])
    print("Pred:", price)
    print("---------------------------")
    selected_indices.append(i)

  0%|          | 11/43961 [00:21<15:24:18,  1.26s/it]

---------------------------
True: 4000000000.0
Pred: 4008560128.0
---------------------------


  0%|          | 29/43961 [00:39<10:31:08,  1.16it/s]

---------------------------
True: 3000000000.0
Pred: 2989390336.0
---------------------------


  0%|          | 74/43961 [01:18<17:23:15,  1.43s/it]

...

---------------------------
True: 7000000000.0
Pred: 6996474880.0
---------------------------


 27%|██▋       | 11856/43961 [2:52:05<7:44:24,  1.15it/s]

---------------------------
True: 4650000000.0
Pred: 4678880768.0
---------------------------


 27%|██▋       | 11967/43961 [2:54:10<7:45:39,  1.15it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-128-d662d4c4bba5>", line 12, in <cell line: 3>
    prices = predict(model, scaler, input_encoding)
  File "<ipython-input-14-6bca2d629542>", line 4, in predict
    logits = model(**input_encoding)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1527, in _call_impl
    return forward_call(*args, **kwargs)
  File "/content/drive/MyDrive/AI/DS/modeling.py", line 43, in forward
    encoder_outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._

In [ ]:
torch.save(selected_indices, "/content/selected_indices.pt")

In [ ]:
np.abs(9_445_432_576 - 9_345_432_576)/9345432576

0.010700414259775406

In [ ]:
selected_indices

In [ ]:
input_encoding = transform_input(tokenizer, input_dict)
prices = predict(model, scaler, input_encoding)

prices

[[6749865472.0]]

In [ ]:
np.exp(np.log(8199999999/1000000))

8199.999999000005

In [ ]:
np.log(7229101056/1000000)

8.88586997215283

In [ ]:
Ground Truth: 15000000000
Prediction  : 18185609216

In [ ]:
def normalized_label(scaler, labels, mode=None):
  if mode == "test":
    return [None]*len(labels)

  labels = np.asarray(labels)
  labels = labels[:, None]

  return scaler.transform(labels)

def merge_and_encode_input(tokenizer, data):
  inputs = []
  for i in tqdm(range(len(data["price"]))):
    input_text = f"Loại: {data['estate_type'][i]}{tokenizer.eos_token}Diện tích: {data['square'][i]}{tokenizer.eos_token}Mô tả: {data['description'][i]}"

    inputs.append(tokenizer(input_text, return_attention_mask=False).input_ids)

  return inputs

def filter_length(data, max_ids_length):
  filtered_ids = []
  filtered_labels = []
  for _, (ids, label) in enumerate(tqdm(zip(data["input_ids"], data["labels"]))):
    if len(ids) <= max_ids_length:
      filtered_ids.append(ids)
      filtered_labels.append(label)

  return Dataset.from_dict({
      "input_ids": filtered_ids,
      "labels": filtered_labels
  })


def process_data(tokenizer, data, max_ids_length=1024, mode=None):
  scaler = torch.load("scaler.pt")

  encoding_data = {"input_ids": merge_and_encode_input(tokenizer, data),
                "labels": normalized_label(scaler, data["price"], mode)}
  encoding_data = filter_length(encoding_data, max_ids_length)
  # validation set

  return {
      "scaler": scaler,
      "data": encoding_data
  }


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

tokenizer_config.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

In [ ]:
scaled_data = process_data(tokenizer, train_dict, validation_dict, test_dict)

100%|██████████| 199911/199911 [03:06<00:00, 1069.61it/s]
199911it [00:00, 1103547.47it/s]
100%|██████████| 43961/43961 [01:06<00:00, 656.83it/s]
43961it [00:00, 308922.76it/s]
100%|██████████| 6924/6924 [00:09<00:00, 746.00it/s]
6924it [00:00, 371467.91it/s]


In [ ]:
torch.save(scaled_data["scaler"], "scaler.pt")

In [ ]:
scaled_data["scaler"]

NameError: ignored

In [9]:
from flask import Flask, request, jsonify
import torch
import torch.nn as nn
from torchvision import transforms
# from your_data_preprocessing_module import preprocess_input_data, postprocess_output

app = Flask(__name__)

# Định nghĩa mô hình PyTorch của bạn
# class YourModel(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size):
#         super(YourModel, self).__init__()
#         # Định nghĩa kiến trúc mạng nơ-ron của bạn ở đây

#     def forward(self, x):
#         # Định nghĩa quá trình lan truyền thuận của mạng nơ-ron của bạn ở đây
#         return x

# # Khởi tạo mô hình
# model = YourModel(input_size, hidden_size, output_size)
# model.load_state_dict(torch.load('your_model_path.pth'))  # Đường dẫn đến trọng số đã được lưu

# # Chuyển mô hình sang chế độ đánh giá
# model.eval()

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()

    # Xử lý dữ liệu đầu vào
    input_data = preprocess_input_data(data['type'], data['area'], data['description'])

    # Chuyển đổi input_data thành tensor PyTorch
    input_tensor = torch.Tensor(input_data)

    # Dự đoán giá
    with torch.no_grad():
        prediction = model(input_tensor)

    # Chuyển đổi tensor PyTorch thành giá trị có thể trả về
    output_price = postprocess_output(prediction.numpy())

    return jsonify({'price': output_price})

if __name__ == '__main__':
    print("run")
    app.run(port=5000)

!sudo lsof -i -P -n

run
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


COMMAND   PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
node        7 root   21u  IPv6  18971      0t0  TCP *:8080 (LISTEN)
node        7 root   27u  IPv4  24222      0t0  TCP 172.28.0.12:52088->172.28.0.12:6000 (ESTABLISHED)
node        7 root   28u  IPv6 108026      0t0  TCP 172.28.0.12:8080->172.28.0.1:57554 (ESTABLISHED)
kernel_ma  21 root    3u  IPv4  24223      0t0  TCP 172.28.0.12:6000->172.28.0.12:52088 (ESTABLISHED)
kernel_ma  21 root    7u  IPv4  18503      0t0  TCP 172.28.0.12:6000 (LISTEN)
kernel_ma  21 root    8u  IPv4  18249      0t0  TCP 172.28.0.12:57822->172.28.0.12:9000 (ESTABLISHED)
kernel_ma  21 root    9u  IPv4  25065      0t0  TCP 172.28.0.12:47156->172.28.0.12:9000 (ESTABLISHED)
kernel_ma  21 root   10u  IPv4  27078      0t0  TCP 172.28.0.12:6000->172.28.0.1:56130 (ESTABLISHED)
colab-fil  59 root    3u  IPv4  18069      0t0  TCP 127.0.0.1:3453 (LISTEN)
jupyter-n  80 root    7u  IPv4  18239      0t0  TCP 172.28.0.12:9000 (LISTEN)
jupyter-n  80 root    8u  IPv4  1

In [8]:
!sudo lsof -i -P -n

COMMAND   PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
node        7 root   21u  IPv6  18971      0t0  TCP *:8080 (LISTEN)
node        7 root   26u  IPv6 104791      0t0  TCP 172.28.0.12:8080->172.28.0.1:47648 (ESTABLISHED)
node        7 root   27u  IPv4  24222      0t0  TCP 172.28.0.12:52088->172.28.0.12:6000 (ESTABLISHED)
node        7 root   28u  IPv6 104914      0t0  TCP 172.28.0.12:8080->172.28.0.1:47662 (ESTABLISHED)
kernel_ma  21 root    3u  IPv4  24223      0t0  TCP 172.28.0.12:6000->172.28.0.12:52088 (ESTABLISHED)
kernel_ma  21 root    7u  IPv4  18503      0t0  TCP 172.28.0.12:6000 (LISTEN)
kernel_ma  21 root    8u  IPv4  18249      0t0  TCP 172.28.0.12:57822->172.28.0.12:9000 (ESTABLISHED)
kernel_ma  21 root    9u  IPv4  25065      0t0  TCP 172.28.0.12:47156->172.28.0.12:9000 (ESTABLISHED)
kernel_ma  21 root   10u  IPv4  27078      0t0  TCP 172.28.0.12:6000->172.28.0.1:56130 (ESTABLISHED)
colab-fil  59 root    3u  IPv4  18069      0t0  TCP 127.0.0.1:3453 (LISTEN)
jupyter-n 

In [7]:
!curl https://localhost:5000

curl: (7) Failed to connect to localhost port 5000 after 0 ms: Connection refused
